In [2]:
import sqlite3
conn = sqlite3.connect("AQI.db")
with conn:
    sql = '''
    CREATE TABLE IF NOT EXISTS records (
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        sitename TEXT NOT NULL,
        county TEXT,
        aqi INTEGER,
        status TEXT,
        pm25 NUMERIC,
        date TEXT,
        lat NUMERIC,
        lon NUMERIC,
        UNIQUE(sitename,date)
    );
    '''
    cursor = conn.cursor()
    cursor.execute(sql)



In [3]:
import requests

url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    for items in data['records']:
        sitenames.add(items['sitename'])

    sitenames = list(sitenames)
print(sitenames)

['忠明', '馬公', '桃園', '前鎮', '永和', '高雄（湖內）', '橋頭', '中山', '富貴角', '屏東（琉球）', '基隆', '陽明', '左營', '淡水', '新北(樹林)', '前金', '林園', '二林', '板橋', '崙背', '線西', '菜寮', '林口', '臺南（麻豆）', '古亭', '冬山', '大城', '彰化（員林）', '觀音', '湖口', '汐止', '西屯', '臺西', '龍潭', '金門', '屏東', '新竹', '臺東', '善化', '馬祖', '三義', '美濃', '屏東(枋山)', '南投', '彰化', '鳳山', '嘉義', '關山', '宜蘭（頭城）', '大里', '平鎮', '宜蘭', '頭份', '大寮', '新店', '三重', '復興', '新營', '竹東', '竹山', '土城', '安南', '朴子', '花蓮', '恆春', '小港', '新莊', '埔里', '仁武', '大園', '沙鹿', '萬華', '楠梓', '萬里', '斗六', '中壢', '士林', '大同', '苗栗', '潮州', '臺南', '麥寮', '松山', '新港', '豐原']


In [4]:
import datasource
datasource.get_selected_data(sitename='屏東(枋山)')

[]

In [5]:
import sqlite3
conn = sqlite3.connect("AQI.db")
with conn:
    sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
    values ('屏東(枋山)','屏東縣', 34, '良好',18,'2024-11-04 10:00', 22.260899, 120.651472);
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [6]:
import sqlite3
conn = sqlite3.connect("AQI.db")
url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    with conn:
        cursor = conn.cursor()
        for items in data['records']:
            sitename = items['sitename']
            county = items['county']
            aqi = int(items['aqi']) if items['aqi'] != '' else 0
            status = items['status']
            pm25 = float(items['pm2.5']) if items['pm2.5'] != '' else 0.0
            date = items['datacreationdate']
            lon = float(items['longitude']) if items['longitude'] != '' else 0.0
            lat = float(items['latitude']) if items['latitude'] != '' else 0.0
            sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
                    values (?,?, ?, ?,?,?,?,?);
            '''
            cursor.execute(sql,(sitename, county, aqi, status,pm25,date,lat,lon))

        
        

    

In [7]:
conn = sqlite3.connect("AQI.db")
with conn:
    cursor = conn.cursor()
    sql = '''
    SELECT DISTINCT sitename
    FROM records
    '''
    cursor.execute(sql)
    sitenames = []

    for items in cursor.fetchall():
        sitenames.append(items[0])
    
    print(sitenames)

['三義', '三重', '中壢', '中山', '二林', '仁武', '冬山', '前金', '前鎮', '南投', '古亭', '善化', '嘉義', '土城', '埔里', '基隆', '士林', '大同', '大園', '大城', '大寮', '大里', '安南', '宜蘭', '宜蘭（頭城）', '富貴角', '小港', '屏東', '屏東(枋山)', '屏東（琉球）', '崙背', '左營', '平鎮', '彰化', '彰化（員林）', '復興', '忠明', '恆春', '斗六', '新北(樹林)', '新店', '新港', '新營', '新竹', '新莊', '朴子', '松山', '板橋', '林口', '林園', '桃園', '楠梓', '橋頭', '永和', '汐止', '沙鹿', '淡水', '湖口', '潮州', '竹山', '竹東', '線西', '美濃', '臺南', '臺南（麻豆）', '臺東', '臺西', '花蓮', '苗栗', '菜寮', '萬華', '萬里', '西屯', '觀音', '豐原', '金門', '關山', '陽明', '頭份', '馬公', '馬祖', '高雄（湖內）', '鳳山', '麥寮', '龍潭']


In [8]:
conn = sqlite3.connect("AQI.db")
with conn:
    cursor = conn.cursor()        
    sql = '''
    SELECT date,county,aqi,pm25,status,lat,lon
    FROM records
    WHERE sitename=?
    ORDER BY date DESC;
    '''
    cursor.execute(sql,('富貴角',))
    sitename_list = [list(item) for item in cursor.fetchall()]
    print(sitename_list)

[['2024-11-05 09:00', '新北市', 61, 17, '普通', 25.29681695, 121.53656894], ['2024-11-05 08:00', '新北市', 61, 18, '普通', 25.29681695, 121.53656894], ['2024-11-05 07:00', '新北市', 61, 20, '普通', 25.29681695, 121.53656894], ['2024-11-05 06:00', '新北市', 58, 15, '普通', 25.29681695, 121.53656894], ['2024-11-05 05:00', '新北市', 58, 19, '普通', 25.29681695, 121.53656894], ['2024-11-05 04:00', '新北市', 58, 20, '普通', 25.29681695, 121.53656894], ['2024-11-05 03:00', '新北市', 54, 19, '普通', 25.29681695, 121.53656894]]
